# Restaurant Recommender

## Introduction
Due to the long hours of typical engagements, many teams work must work late and order dinner in the office, especially during busy season. 
One common pain point during this time is finding a restaurant to order from. This can be additionally challenging when working at client-sites where the team members are often unfamiliar with the area.

This program facilitates the search for a restaurant with the use of publicly available restaurant REST APIs, saving teams time and energy.

## Methodology
1. Configure the parameters for the restaurant search  
  *  Address of office location 
  *  Radius of search (in miles)  
  *  Keywords to search for  
  *  Maximum # of search results to be returned per API  
2. Geocode the address to obtain latitude, longitude coordinates
3. Access APIs  
4. Parse responses  
5. Combine and Map results  
6. Sort results

## Remaining Items  
*  Zomato API's /search endpoint does not correctly limit search radius 
  *  Searching with a radius below 50 meters results in a search around coordinates (0.00, 0.00)
*  Zomato API's /geocode endpoint does not allow configuration of search radius nor configuration of max search results
*  Currently waiting to hear back from the OpenTable API team regarding access key
*  Currently waiting to hear back from the Trip Expert API team regarding access key
*  Design configuration to utilize user inputs such that non-technical members may use this application as well

In [1]:
import requests
import json
import pandas as pd

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

### Helper Functions

In [2]:
def get_latlon_HERE(app_id, app_code, address=None, postalcode=None, maxresults=1):
    '''
    Geocode provided address
    
    Returns latitude and longitude coordinates
    '''
    parameters = {'app_id': app_id, 'app_code': app_code, 'searchtext': address, 'postalcode': postalcode, 
                  'maxresults': maxresults}
    response = requests.get("https://geocoder.api.here.com/6.2/geocode.json", params = parameters)
    
    lat = json.loads(response.text)['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]['Latitude']
    long = json.loads(response.text)['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]['Longitude']
    
    return (lat,long)

In [3]:
def search_zomato(api_key, lat, lon, radius, sort, order, keyword=None, cuisines=None, category=None, count=20):
    '''
    Utilizes Zomato API's /search endpoint
    
    Returns restaurants around a given latitude, longitude within a given radius
    '''
    parameters = {'lat': lat, 'lon': lon, 'radius': radius, 'q': keyword, 'cuisines': cuisines, 'category': category, 
                  'count': count, 'sort': sort, 'order': order}
    headers = {'Accept': 'application/json', 'user_key': api_key} 
    response = requests.get("https://developers.zomato.com/api/v2.1/search", headers = headers, params = parameters)
    
    return json.loads(response.text)

In [4]:
def search_zomato_geocode(api_key, lat, lon):
    '''
    Utilizes Zomato API's /geocode endpoint
    Search radius cannot be set
    Limit of search results cannot be set
    
    Returns "nearby restaurants" around a given latitude, longitude
    '''
    parameters = {'lat': lat, 'lon': lon}
    headers = {'Accept': 'application/json', 'user_key': api_key}
    response = requests.get("https://developers.zomato.com/api/v2.1/geocode", headers = headers, params = parameters)
    
    return json.loads(response.text)

In [5]:
def search_yelp(api_key, addr, lat, long, radius, sort_by='rating', term='food', categories='food', open_now=True, limit=20):
    '''
    Utilizes Yelp API's /search endpoint
    
    Returns restaurants around a given latitude, longitude within a given radius
    '''
    parameters = {'term': term, 'categories': categories, 'location': addr, 'radius': radius, 'attributes': ['pickup','delivery'],
                  'sort_by': sort_by, 'open_now': open_now, 'limit': limit}
    headers = {'Authorization': 'Bearer %s' % api_key}
    response = requests.get("https://api.yelp.com/v3/businesses/search", headers = headers, params = parameters)
    
    return json.loads(response.text)

In [6]:
def search_eatstreet(api_key, lat, long, radius, method='both', search=None):
    '''
    Utilizes EatStreet API's restaurant/search endpoint
    
    Returns restaurants around a given latitude, longitude within a given radius
    '''
    parameters = {'access-token': api_key, 'latitude': lat, 'longitude': long, 'pickup-radius': radius, 'method': method, 'search': search}
    headers = {'X-Access-Token': '%s' % api_key}
    response = requests.get("https://eatstreet.com/publicapi/v1/restaurant/search", headers = headers, params = parameters)
    
    return json.loads(response.text)   

### Search Parameters

In [7]:
'''
Configure search parameters
'''
MILES_METERS_CONVERSION = 1609.34

origin_address = '300 Madison Ave New York, NY'
origin_postalcode = '10017'

keyword = 'restaurant'
cuisine = None
zomato_category = [1,2,5,6,7,8,9,10,13]
yelp_categories = 'restaurants'
eatstreet_method = 'both' # pickup, delivery, or both

radius_miles = .5
radius_meters = int(radius_miles * MILES_METERS_CONVERSION)

sort = 'rating'
order = 'desc'
limit = 10


if False:
    '''
    Zomato categories:
        1 - Delivery
        2 - Dine-out
        3 - Nightlife
        4 - Catching-up
        5 - Takeaway
        6 - Cafes
        7 - Daily Menus
        8 - Breakfast
        9 - Lunch
        10 - Dinner
        11 - Pubs and Bars
        13 - Pocket Friendly Delivery
        14 - Clubs & Lounges
    '''

### Geocoding Address

In [8]:
# HERE API: 250,000 calls/month
HERE_APP_ID = 'AIjEx3suCWa026cEht8M'
HERE_APP_CODE = 'K0viSXXYzW5bjf9q7XAwCQ'

(origin_lat,origin_long) = get_latlon_HERE(HERE_APP_ID, HERE_APP_CODE, address = origin_address, postalcode = origin_postalcode)

In [9]:
print(origin_lat,origin_long)

40.75246 -73.97954


### API Requests

#### Zomato

In [10]:
'''
Send request to Zomato API
https://developers.zomato.com/documentation#!/restaurant/search
'''
# ZOMATO: 1,000 calls/day
ZOMATO_API_KEY = '00de557adb2de31c4e3ddd4e1143ba49'

z_response = search_zomato(ZOMATO_API_KEY, origin_lat, origin_long, radius_meters, sort, order, keyword = keyword, 
                           category = zomato_category, count=limit)

num_z_results = z_response['results_shown']
print("{} Zomato result(s) returned.\n".format(num_z_results))

10 Zomato result(s) returned.



In [11]:
'''
Send request to Zomato API
https://developers.zomato.com/documentation#!/restaurant/search
'''
# ZOMATO: 1,000 calls/day
ZOMATO_API_KEY = '00de557adb2de31c4e3ddd4e1143ba49'

z_response_geocode = search_zomato_geocode(ZOMATO_API_KEY, origin_lat, origin_long)

num_zgeo_results = len(z_response_geocode['nearby_restaurants'])
print("{} Zomato (geocode) result(s) returned.\n".format(num_z_results))

10 Zomato (geocode) result(s) returned.



In [12]:
'''
Parse Zomato response
'''
z_response = z_response['restaurants']
#z_response = z_response_geocode['nearby_restaurants'] # if using search_zomato_geocode

zomato_results = pd.DataFrame(columns = ['Source', 'Ranking','Name', 'URL', 'Categories', 'Rating', '# of Ratings', 
                                         'Address', 'Latitude', 'Longitude'])

#num_z_results = num_zgeo_results # if using search_zomato_geocode
for idx in range(num_z_results):
    zomato_results.loc[idx, 'Name'] = z_response[idx]['restaurant']['name']
    zomato_results.loc[idx, 'URL'] = z_response[idx]['restaurant']['url']
    #menu_url = z_response[idx]['restaurant']['menu_url']
    #photos_url = z_response[idx]['restaurant']['photos_url']
    zomato_results.loc[idx, 'Categories'] = z_response[idx]['restaurant']['cuisines']
    zomato_results.loc[idx, 'Rating'] = z_response[idx]['restaurant']['user_rating']['aggregate_rating']
    zomato_results.loc[idx, '# of Ratings'] = z_response[idx]['restaurant']['user_rating']['votes']
    zomato_results.loc[idx, 'Address'] = z_response[idx]['restaurant']['location']['address']
    zomato_results.loc[idx, 'Latitude'] = z_response[idx]['restaurant']['location']['latitude']
    zomato_results.loc[idx, 'Longitude'] = z_response[idx]['restaurant']['location']['longitude']
    zomato_results.loc[idx, 'Ranking'] = idx + 1
    
zomato_results['Source'] = 'Zomato'

zomato_results.head(10)                                         

,Source,Ranking,Name,URL,Categories,Rating,# of Ratings,Address,Latitude,Longitude
0,Zomato,1,Katz's Delicatessen,https://www.zomato.com/new-york-city/katzs-del...,Sandwich,4.9,2504,"205 East Houston Street, New York 10002",40.7223277778,-73.9873500000
1,Zomato,2,Buddakan,https://www.zomato.com/new-york-city/buddakan-...,"Chinese, Fusion, Asian",4.9,907,75 9th Avenue 10011,40.7422762672,-74.0048000962
2,Zomato,3,Peter Luger Steak House,https://www.zomato.com/new-york-city/peter-lug...,"Steak, American, German, Burger",4.7,1116,"178 Broadway, Brooklyn 11211",40.7098777778,-73.9623416667
3,Zomato,4,Carmine's Italian Restaurant,https://www.zomato.com/new-york-city/carmines-...,"Italian, Southern",4.8,1029,"200 West 44th Street, btwn Broadway & 8th Aven...",40.7579000000,-73.9875700000
4,Zomato,5,Junior's Restaurant,https://www.zomato.com/new-york-city/juniors-r...,"American, Desserts",4.7,531,1515 Broadway--Between Broadway and 8th Avenue...,40.7579300000,-73.9864000000
5,Zomato,6,Forlini's Restaurant,https://www.zomato.com/new-york-city/forlinis-...,Italian,4.7,66,"93 Baxter Street, New York 10013",40.7169694444,-73.9996805556
6,Zomato,7,Max Brenner New York,https://www.zomato.com/new-york-city/max-brenn...,"New American, Desserts",4.7,668,841 Broadway 10003,40.7342000000,-73.9909800000
7,Zomato,8,Sea Restaurant,https://www.zomato.com/new-york-city/sea-resta...,"Asian, Thai",4.6,613,"114 N 6th Street, Brooklyn 11249",40.7182055556,-73.9598388889
8,Zomato,9,Sanford's Restaurant,https://www.zomato.com/new-york-city/sanfords-...,New American,4.6,123,"30-13 Broadway, Queens 11106",40.7621944444,-73.9257583333
9,Zomato,10,Malecón Restaurant,https://www.zomato.com/new-york-city/malecón-r...,"Dominican, Latin American",4.6,100,4141 Broadway 10033,40.8463472222,-73.9385000000


#### Yelp

In [13]:
'''
Send request to Yelp API
https://www.yelp.com/developers/documentation/v3/get_started
'''
# Yelp: 5,000 credits/day
YELP_CLIENT_ID = 'kuRZJ2Xa6JDPVVXdlt6DJA'
YELP_API_KEY = 'f5w858kIw3CC2jBY_W4591TrLPjZwAl8iEGaoRzBmE9VRAVjCEkrWEZCUROuKuxBNAH0DP1AcuavnBCt4eULbvSnQ8EfYUDPMGpFWT6jWrOCjCIgEBzpZaDKMHPDXHYx'

# Note: Yelp's sort takes into account number of reviews as well, not strictly average rating amount
yelp_response = search_yelp(YELP_API_KEY, origin_address, origin_lat, origin_long, radius_meters, sort_by = sort, term = keyword, 
                            categories = yelp_categories, limit = limit)

num_y_results = len(yelp_response['businesses'])
print("{} Yelp result(s) returned.\n".format(num_y_results))

10 Yelp result(s) returned.



In [14]:
'''
Parse Yelp response
'''
yelp_results = pd.DataFrame(columns = ['Source','Ranking','Name','URL','Categories','Rating','# of Ratings',
                                       'Address','Latitude','Longitude'])

for idx in range(num_y_results):
    yelp_results.loc[idx,'Name'] = yelp_response['businesses'][idx]['name']
    yelp_results.loc[idx,'URL'] = yelp_response['businesses'][idx]['url']
    yelp_results.loc[idx,'Rating'] = yelp_response['businesses'][idx]['rating']
    yelp_results.loc[idx,'# of Ratings'] = yelp_response['businesses'][idx]['review_count']
    yelp_results.loc[idx,'Address'] = yelp_response['businesses'][idx]['location']['address1'] + ' ' + \
                                        yelp_response['businesses'][idx]['location']['city'] + ', ' + \
                                        yelp_response['businesses'][idx]['location']['state'] + ' ' + \
                                        yelp_response['businesses'][idx]['location']['zip_code']
    yelp_results.loc[idx,'Latitude'] = yelp_response['businesses'][idx]['coordinates']['latitude']
    yelp_results.loc[idx,'Longitude'] = yelp_response['businesses'][idx]['coordinates']['longitude']
    yelp_results.loc[idx, 'Ranking'] = idx + 1
    
    categories = []
    for i in range(len(yelp_response['businesses'][idx]['categories'])):
        categories.append(yelp_response['businesses'][idx]['categories'][i]['title'])
    yelp_results.loc[idx,'Categories'] = categories
    
yelp_results['Source'] = 'Yelp'

yelp_results

,Source,Ranking,Name,URL,Categories,Rating,# of Ratings,Address,Latitude,Longitude
0,Yelp,1,Royal Grill Halal Food,https://www.yelp.com/biz/royal-grill-halal-foo...,"[Halal, Food Trucks, Street Vendors]",5,261,"44th St And 6th Ave New York, NY 10036",40.7561,-73.9832
1,Yelp,2,Sushi by Bou Times Square,https://www.yelp.com/biz/sushi-by-bou-times-sq...,"[Sushi Bars, Japanese]",4.5,368,"132 W 47th St New York, NY 10036",40.7585,-73.9832
2,Yelp,3,Anytime,https://www.yelp.com/biz/anytime-new-york?adju...,"[Korean, American (New), Tapas/Small Plates]",4.5,369,"23 W 32nd St New York, NY 10001",40.7478,-73.9867
3,Yelp,4,Coney Shack,https://www.yelp.com/biz/coney-shack-new-york-...,"[Food Trucks, Mexican]",4.5,270,"42nd St Times Square New York, NY 10036",40.7553,-73.9866
4,Yelp,5,Stone Bridge Pizza & Salad,https://www.yelp.com/biz/stone-bridge-pizza-an...,"[Pizza, Salad, Food Delivery Services]",4.5,225,"16 E 41st St New York, NY 10017",40.7522,-73.9806
5,Yelp,6,Del Frisco's Double Eagle Steakhouse,https://www.yelp.com/biz/del-friscos-double-ea...,"[Steakhouses, American (New), Seafood]",4.5,2764,"1221 Ave Of The Americas New York, NY 10020",40.7594,-73.9822
6,Yelp,7,Trademark Taste + Grind,https://www.yelp.com/biz/trademark-taste-grind...,"[Venues & Event Spaces, Breakfast & Brunch, Am...",4.5,574,"38 W 36th St New York, NY 10018",40.7503,-73.9855
7,Yelp,8,Royal 35 Steakhouse,https://www.yelp.com/biz/royal-35-steakhouse-n...,[Steakhouses],4.5,165,"1 E 35th St New York, NY 10016",40.749,-73.9835
8,Yelp,9,Gabriel Kreuther,https://www.yelp.com/biz/gabriel-kreuther-new-...,"[Lounges, French]",4.5,367,"41 W 42nd St New York, NY 10036",40.7545,-73.9826
9,Yelp,10,Bubo,https://www.yelp.com/biz/bubo-new-york?adjust_...,"[Tapas/Small Plates, Cocktail Bars, Breakfast ...",4.5,144,"515 3rd Ave New York, NY 10016",40.746,-73.9775


#### EatStreet

In [15]:
'''
Send request to EatStreet API
https://developers.eatstreet.com/endpoint/search
'''
# EatStreet: 100,000 calls/day
EATSTREET_API_KEY = '4c2229f98e1a7aaa'

# Note: There is no 'limit' parameter for EatStreet responses
eatstreet_response = search_eatstreet(EATSTREET_API_KEY, origin_lat, origin_long, radius_miles, method = eatstreet_method, search = keyword)

num_e_results = len(eatstreet_response['restaurants'])
print("{} EatStreet result(s) returned.\n".format(num_e_results))

3 EatStreet result(s) returned.



In [16]:
'''
Parse EatStreet response
'''
eatstreet_results = pd.DataFrame(columns = ['Source','Ranking','Name','URL','Categories','Rating','# of Ratings',
                                            'Address','Latitude','Longitude'])

for idx in range(num_e_results):
    eatstreet_results.loc[idx,'Name'] = eatstreet_response['restaurants'][idx]['name']
    eatstreet_results.loc[idx,'URL'] = eatstreet_response['restaurants'][idx]['url']
    eatstreet_results.loc[idx,'Address'] = eatstreet_response['restaurants'][idx]['streetAddress'] + ' ' + \
                                            eatstreet_response['restaurants'][idx]['city'] + ', ' + \
                                            eatstreet_response['restaurants'][idx]['state'] + ' ' + \
                                            eatstreet_response['restaurants'][idx]['zip']
    eatstreet_results.loc[idx,'Latitude'] = eatstreet_response['restaurants'][idx]['latitude']
    eatstreet_results.loc[idx,'Longitude'] = eatstreet_response['restaurants'][idx]['longitude']
    eatstreet_results.loc[idx,'Categories'] = eatstreet_response['restaurants'][idx]['foodTypes']
    eatstreet_results.loc[idx,'Ranking'] = 'N/A' # Note: EatStreet results are not sorted as EatStreet does not include user ratings
    eatstreet_results.loc[idx,'Rating'] = 'N/A'
    eatstreet_results.loc[idx,'# of Ratings'] = 'N/A'
    
eatstreet_results['Source'] = 'EatStreet'

eatstreet_results

,Source,Ranking,Name,URL,Categories,Rating,# of Ratings,Address,Latitude,Longitude
0,EatStreet,N/A,Ali Baba's Restaurant,https://eatstreet.com/new-york-city-ny/restaur...,"[Turkish Food, Mediterranean Food, Turkish, Me...",N/A,N/A,"862 2nd Ave New York, NY 10017",40.7538,-73.9736
1,EatStreet,N/A,Minar Indian Restaurant - W. 46th St.,https://eatstreet.com/new-york-city-ny/restaur...,"[Indian Food, ]",N/A,N/A,"138 W 46th St New York, NY 10036",40.758,-73.984
2,EatStreet,N/A,Taam Tov Restaurant,https://eatstreet.com/new-york-city-ny/restaur...,"[Middle Eastern Food, Kosher, Middle Eastern, ...",N/A,N/A,"41 W 47th St New York, NY 10036",40.7577,-73.9805


#### OpenTable

In [17]:
# OpenTable: request for API key pending; awaiting response
# https://platform.opentable.com/documentation/#platform-basics

#### Trip Expert

In [18]:
# TripExpert: request for API key pending; awaiting response
# https://www.tripexpert.com/api

### Map Restaurants

In [19]:
'''
Combine results into one dataframe
'''
tables = [zomato_results, yelp_results, eatstreet_results]
all_results = pd.concat(tables, sort=False)

all_results.reset_index(inplace=True)
all_results.drop('index', axis=1, inplace=True)

In [20]:
'''
Map all restaurants
'''
# Cast coordinates to numeric
all_results['Latitude'] = pd.to_numeric(all_results['Latitude'])
all_results['Longitude'] = pd.to_numeric(all_results['Longitude'])

map_origin = folium.Map(location=[origin_lat, origin_long], zoom_start=15)

# add markers to map
for lat, lng, name, source, rating in zip(all_results['Latitude'], all_results['Longitude'], 
                                           all_results['Name'], all_results['Source'], all_results['Rating']):

    label = '{}: {} ({})'.format(name, rating, source)
    label = folium.Popup(label, parse_html=True)
    if source == 'Zomato':
        folium.Marker(
            [lat, lng],
            icon=folium.Icon(color='red',icon='info-sign'),
            popup=label).add_to(map_origin)
    elif source == 'Yelp':
        folium.Marker(
            [lat, lng],
            icon=folium.Icon(color='blue',icon='info-sign'),
            popup=label).add_to(map_origin)
    elif source == 'EatStreet':
        folium.Marker(
            [lat, lng],
            icon=folium.Icon(color='green',icon='info-sign'),
            popup=label).add_to(map_origin)
    
map_origin

In [21]:
'''
Sort all instances based on Ratings and # of Ratings
'''
# Cast to numeric
for idx in range(all_results.shape[0]):
    if all_results.loc[idx,'Rating'] == 'N/A':
        all_results.loc[idx,'Rating'] = None
        all_results.loc[idx,'# of Ratings'] = None

all_results['Rating'] = pd.to_numeric(all_results['Rating'])
all_results['# of Ratings'] = pd.to_numeric(all_results['# of Ratings'])
 
# Sort
all_results.sort_values(['# of Ratings'], ascending=False, inplace=True)
all_results.sort_values(['Rating'], ascending=False, inplace=True)

all_results

,Source,Ranking,Name,URL,Categories,Rating,# of Ratings,Address,Latitude,Longitude
10,Yelp,1,Royal Grill Halal Food,https://www.yelp.com/biz/royal-grill-halal-foo...,"[Halal, Food Trucks, Street Vendors]",5.0,261.0,"44th St And 6th Ave New York, NY 10036",40.756140,-73.983160
1,Zomato,2,Buddakan,https://www.zomato.com/new-york-city/buddakan-...,"Chinese, Fusion, Asian",4.9,907.0,75 9th Avenue 10011,40.742276,-74.004800
0,Zomato,1,Katz's Delicatessen,https://www.zomato.com/new-york-city/katzs-del...,Sandwich,4.9,2504.0,"205 East Houston Street, New York 10002",40.722328,-73.987350
3,Zomato,4,Carmine's Italian Restaurant,https://www.zomato.com/new-york-city/carmines-...,"Italian, Southern",4.8,1029.0,"200 West 44th Street, btwn Broadway & 8th Aven...",40.757900,-73.987570
5,Zomato,6,Forlini's Restaurant,https://www.zomato.com/new-york-city/forlinis-...,Italian,4.7,66.0,"93 Baxter Street, New York 10013",40.716969,-73.999681
2,Zomato,3,Peter Luger Steak House,https://www.zomato.com/new-york-city/peter-lug...,"Steak, American, German, Burger",4.7,1116.0,"178 Broadway, Brooklyn 11211",40.709878,-73.962342
6,Zomato,7,Max Brenner New York,https://www.zomato.com/new-york-city/max-brenn...,"New American, Desserts",4.7,668.0,841 Broadway 10003,40.734200,-73.990980
4,Zomato,5,Junior's Restaurant,https://www.zomato.com/new-york-city/juniors-r...,"American, Desserts",4.7,531.0,1515 Broadway--Between Broadway and 8th Avenue...,40.757930,-73.986400
7,Zomato,8,Sea Restaurant,https://www.zomato.com/new-york-city/sea-resta...,"Asian, Thai",4.6,613.0,"114 N 6th Street, Brooklyn 11249",40.718206,-73.959839
9,Zomato,10,Malecón Restaurant,https://www.zomato.com/new-york-city/malecón-r...,"Dominican, Latin American",4.6,100.0,4141 Broadway 10033,40.846347,-73.938500
